This script reads data in from 5 datasets taken from the ABS website.  
It sorts this data and assembles it into a master dataframe which can be used for further analysis.

The CSV datasets were found at https://explore.data.abs.gov.au/.  
* Census 2021, G36 Dwelling structure, Postal Areas (POA)  
* Census 2021, G41 Dwelling structure by number of bedrooms, Postal Areas (POA)  
* Census 2021, G42 Dwelling structure by household composition and family composition, Postal Areas (POA)  
* Census 2021, G19 Type of long-term health condition by age by sex, Postal Areas (POA)  
* Census 2021, G01 Selected person characteristics by sex, Postal Areas (POA) *for reference data only*



In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

In [2]:
# Read in CSVs
# G41 Dwelling structure by number of bedrooms
bedrooms_path = "Resources/ABS_C21_G41_POA_1.0.0.csv"
bedrooms_df = pd.read_csv(bedrooms_path)

# G42 Dwelling structure by household composition and family composition
comp_path = "Resources/ABS_C21_G42_POA_1.0.0.csv"
comp_df = pd.read_csv(comp_path)

# G19 Type of long-term health condition
mh_path = "Resources/ABS_C21_G19_POA_1.0.0_3.91._T....csv"
mh_df = pd.read_csv(mh_path)

# G01 Reference data only
ref_path = "Resources/ABS_C21_G01_POA_1.0.0_3.P_1....csv"
ref_df = pd.read_csv(ref_path)

In [3]:
bedrooms_df["STRD: Dwelling structure"].unique()

array(['32: Flat or apartment in a three storey block',
       '2: Semi-detached, row or terrace house, townhouse etc: Total',
       '31: Flat or apartment in a one or two storey block',
       '_N: Not stated', '9: Other dwelling',
       '3: Flat or apartment: Total',
       '35: Flat or apartment attached to a house', '11: Separate house',
       '22: Semi-detached, row or terrace house, townhouse etc. with two or more storeys',
       '_T: Total',
       '21: Semi-detached, row or terrace house, townhouse etc. with one storey',
       '34: Flat or apartment in a nine or more storey block',
       '33: Flat or apartment in a four to eight storey block'],
      dtype=object)

# Dwelling Type by Postcode

In [4]:
# Convert region column to postcode
bedrooms_df["Postcode"] = bedrooms_df['REGION: Region'].astype(str).str[0:4]

# Filter to dwellings only, totals only for dwelling types and remove non-postcode column
filt_bedrooms_df = bedrooms_df.loc[((bedrooms_df["STRD: Dwelling structure"] == "11: Separate house") |
                (bedrooms_df["STRD: Dwelling structure"] == "2: Semi-detached, row or terrace house, townhouse etc: Total") |
                (bedrooms_df["STRD: Dwelling structure"] == "3: Flat or apartment: Total") |
                (bedrooms_df["STRD: Dwelling structure"] == "9: Other dwelling")) &
                (bedrooms_df["Postcode"] != "9797"), :]

# Filter to required columns only, rename columns and return dwelling type count value as integers
filt_bedrooms_df = filt_bedrooms_df[["STRD: Dwelling structure", "Postcode", "STATE: State", "OBS_VALUE"]]
filt_bedrooms_df = filt_bedrooms_df.rename(columns={"OBS_VALUE": "Dwelling Type", "STRD: Dwelling structure": "Count"})
filt_bedrooms_df["Dwelling Type"] = filt_bedrooms_df["Dwelling Type"].astype(int)

# Pivot dataframe by Postcode
dwell_poa_df = filt_bedrooms_df.groupby(["Postcode", "Count"]).sum().unstack()

# Rename dwelling type columns
dwell_poa_df = dwell_poa_df.rename(columns={"11: Separate house": "House",
                                "2: Semi-detached, row or terrace house, townhouse etc: Total": "Semi-detached",
                                "3: Flat or apartment: Total": "Apartment",
                                "9: Other dwelling": "Other dwelling"})
dwell_poa_df

Dwelling Type                                       
Count            House Semi-detached Apartment Other dwelling
Postcode                                                     
0800                33             6      5876              4
0810             14828          2642      5649             85
0812              9601          1127      1549            134
0820              4926          3337      6733            104
0822              9415           655       168            348
...                ...           ...       ...            ...
7466                27             0         0              0
7467              1572            25        31             14
7468               557             6         8              0
7469               607            62         0              8
7470               598            22         0              6

[2641 rows x 4 columns]

# Number of Bedrooms by Postcode

In [5]:
# Convert region column to postcode
bedrooms_df["Postcode"] = bedrooms_df['REGION: Region'].astype(str).str[0:4]

# Filter to columns with bedroom counts only and remove non-postcode row
filt_bedrooms_df = bedrooms_df.loc[(bedrooms_df["BEDD: Number of bedrooms in private dwelling"] != '_N: Number of bedrooms not stated') &
                    (bedrooms_df["BEDD: Number of bedrooms in private dwelling"] != '_T: Total') &
                    (bedrooms_df["Postcode"] != "9797"), :]

# Filter to required columns only and rename columns
filt_bedrooms_df = filt_bedrooms_df[["BEDD: Number of bedrooms in private dwelling", "Postcode", "OBS_VALUE"]]
filt_bedrooms_df = filt_bedrooms_df.rename(columns={"OBS_VALUE": "Number of Bedrooms", "BEDD: Number of bedrooms in private dwelling": "Count"})

# Pivot dataframe by Postcode
bedrooms_poa_df = filt_bedrooms_df.groupby(["Postcode", "Count"]).sum().unstack()

# Rename number of bedrooms columns
bedrooms_poa_df = bedrooms_poa_df.rename(columns={"0: None (includes studio apartments or bedsitters)": "No bedrooms (studios, etc)",
                                "1: 1 bedroom": "1 bedroom",
                                "2: 2 bedrooms": "2 bedrooms",
                                "3: 3 bedrooms": "3 bedrooms",
                                "4: 4 bedrooms": "4 bedrooms",
                                "5: 5 bedrooms": "5 bedrooms",
                                "6: 6 or more bedrooms": "6 or more bedrooms"})
bedrooms_poa_df


Number of Bedrooms                                  \
Count    No bedrooms (studios, etc) 1 bedroom 2 bedrooms 3 bedrooms   
Postcode                                                              
0800                            172      1856       3749       2805   
0810                            207      1768       8552       9360   
0812                             67       448       3389       6590   
0820                            167      1463       7566       7580   
0822                            216       873       3083       4392   
...                             ...       ...        ...        ...   
7466                              0         0         10         12   
7467                              5        84        319        987   
7468                              0        33        145        251   
7469                              3        62        126        412   
7470                             14        18        114        458   

                                                   
Count    4 bedrooms 5 bedrooms 6 or more bedrooms  
Postcode                                           
0800            168         12                  6  
0810           5484       1260                311  
0812           2353        543                 95  
0820           2372        592                168  
0822           1811        344                183  
...             ...        ...                ...  
7466              0          0                  0  
7467            218         26                 10  
7468            116          6                 14  
7469             80         14                  0  
7470             50          6                  3  

[2641 rows x 7 columns]

# Household Composition by Postcode

In [6]:
# Convert region column to postcode
comp_df["Postcode"] = comp_df['REGION: Region'].astype(str).str[0:4]

# Filter to columns with household composition counts only and remove non-postcode row
filt_comp_df = comp_df.loc[(comp_df["HHCFMCD: Household and Family composition"] != '5: Family household: Total') &
                    (comp_df["HHCFMCD: Household and Family composition"] != '_T: Total') &
                    (comp_df["Postcode"] != "9797"), :]

# Filter to required columns only and rename columns
filt_comp_df = filt_comp_df[["HHCFMCD: Household and Family composition", "Postcode", "OBS_VALUE"]]
filt_comp_df = filt_comp_df.rename(columns={"OBS_VALUE": "Household Composition", "HHCFMCD: Household and Family composition": "Count"})

# Pivot dataframe by Postcode
comp_poa_df = filt_comp_df.groupby(["Postcode", "Count"]).sum().unstack()

# Rename number of bedrooms columns
comp_poa_df = comp_poa_df.rename(columns={"1: Family household: Couple family with no children": "Couple with no children",
                                "2: Family household: Couple family with children": "Couple with children",
                                "3: Family household: One parent family": "One parent with children",
                                "4: Family household: Other family": "Other family",
                                "6: Lone person household": "Sole person household",
                                "7: Group household": "Group household"})

comp_poa_df

Household Composition                       \
Count    Couple with no children Couple with children   
Postcode                                                
0800                        2874                 1181   
0810                        6553                 8881   
0812                        3237                 4486   
0820                        5792                 4755   
0822                        2456                 4369   
...                          ...                  ...   
7466                           6                   10   
7467                         450                  272   
7468                         181                  114   
7469                         188                  110   
7470                         181                   93   

                                                                      \
Count    One parent with children Other family Sole person household   
Postcode                                                               
0800                          395          134                  3420   
0810                         2893          425                  6996   
0812                         2032          204                  3249   
0820                         1644          220                  5987   
0822                         1989          404                  1921   
...                           ...          ...                   ...   
7466                            0            0                     6   
7467                          170            8                   731   
7468                           38            0                   229   
7469                           74            0                   305   
7470                           57           10                   303   

                          
Count    Group household  
Postcode                  
0800                 910  
0810                1737  
0812                 592  
0820                1843  
0822                 268  
...                  ...  
7466                   8  
7467                  48  
7468                  18  
7469                  15  
7470                  30  

[2641 rows x 6 columns]

# Mental Health Condition by Postcode

In [7]:
# Convert region column to postcode
mh_df["Postcode"] = mh_df['REGION: Region'].astype(str).str[0:4]

# Filter to remove non-postcode rows
filt_mh_df = mh_df.loc[(mh_df["Postcode"] != "9494") & 
                    (mh_df["Postcode"] != "9797"), :]

# Filter to required columns only and rename columns
filt_mh_df = filt_mh_df[["Postcode", "OBS_VALUE"]]
filt_mh_df = filt_mh_df.rename(columns={"OBS_VALUE": "People with Mental Health Conditions"})

mh_poa_df = filt_mh_df.set_index("Postcode")
mh_poa_df

,People with Mental Health Conditions
Postcode,
3188,1422
6640,18
2429,1131
3523,463
5251,2156
...,...
4496,7
6052,1107
4364,6


# Reference Data

In [8]:
# Convert region column to postcode
ref_df["Postcode"] = ref_df['REGION: Region'].astype(str).str[0:4]

# Filter to remove non-postcode rows
filt_ref_df = ref_df.loc[(ref_df["Postcode"] != "9494") & 
                    (ref_df["Postcode"] != "9797"), :]

# Filter to required columns only and rename columns
filt_ref_df = filt_ref_df[["Postcode", "OBS_VALUE", "STATE: State"]]
filt_ref_df = filt_ref_df.rename(columns={"OBS_VALUE": "Total People", "STATE: State": "State"})

ref_poa_df = filt_ref_df.set_index("Postcode")
ref_poa_df

,Total People,State
Postcode,,
2711,3197,1: New South Wales
3289,804,2: Victoria
3659,748,2: Victoria
4205,6333,3: Queensland
4486,669,3: Queensland
...,...,...
3646,601,2: Victoria
4655,66789,3: Queensland
4806,3771,3: Queensland


# Master Dataframe

In [9]:
# Create master dataframe from multitier dfs
master_df = pd.concat([dwell_poa_df, bedrooms_poa_df,comp_poa_df], axis=1)

# Remove multitiers
master_df.columns.names = ["",""]
master_df.columns = master_df.columns.droplevel(0)

# Add single tier dfs with 
master_df = pd.concat([master_df, mh_poa_df, ref_poa_df], axis=1)

# Rename index
master_df.index.names = ["Postcode"]
master_df

,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,5 bedrooms,6 or more bedrooms,Couple with no children,Couple with children,One parent with children,Other family,Sole person household,Group household,People with Mental Health Conditions,Total People,State
Postcode,,,,,,,,,,,,,,,,,,,,
0800,33,6,5876,4,172,1856,3749,2805,168,12,6,2874,1181,395,134,3420,910,344,7149,7: Northern Territory
0810,14828,2642,5649,85,207,1768,8552,9360,5484,1260,311,6553,8881,2893,425,6996,1737,1971,34330,7: Northern Territory
0812,9601,1127,1549,134,67,448,3389,6590,2353,543,95,3237,4486,2032,204,3249,592,1076,18634,7: Northern Territory
0820,4926,3337,6733,104,167,1463,7566,7580,2372,592,168,5792,4755,1644,220,5987,1843,1112,19920,7: Northern Territory
0822,9415,655,168,348,216,873,3083,4392,1811,344,183,2456,4369,1989,404,1921,268,755,25304,7: Northern Territory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7466,27,0,0,0,0,0,10,12,0,0,0,6,10,0,0,6,8,7,32,6: Tasmania
7467,1572,25,31,14,5,84,319,987,218,26,10,450,272,170,8,731,48,234,1808,6: Tasmania
7468,557,6,8,0,0,33,145,251,116,6,14,181,114,38,0,229,18,37,697,6: Tasmania


In [10]:
"""
# Add calculated columns
# Column summing dwelling types
master_df["Sum of Dwelling Types"] = master_df["House"] + master_df["Semi-detached"] + master_df["Apartment"] + master_df["Other dwelling"]
# Column summing number of bedrooms
master_df["Sum of Bedrooms"] = master_df["No bedrooms (studios, etc)"] + master_df["1 bedroom"] + master_df["2 bedrooms"] + master_df["3 bedrooms"] + master_df["4 bedrooms"] + master_df["5 bedrooms"] + master_df["6 or more bedrooms"]
# Column summing house composition
master_df["Sum of House Composition"] = master_df["Couple with no children"] + master_df["Couple with children"] + master_df["One parent with children"] + master_df["Sole person household"] + master_df["Group household"]
master_df
"""

'\n# Add calculated columns\n# Column summing dwelling types\nmaster_df["Sum of Dwelling Types"] = master_df["House"] + master_df["Semi-detached"] + master_df["Apartment"] + master_df["Other dwelling"]\n# Column summing number of bedrooms\nmaster_df["Sum of Bedrooms"] = master_df["No bedrooms (studios, etc)"] + master_df["1 bedroom"] + master_df["2 bedrooms"] + master_df["3 bedrooms"] + master_df["4 bedrooms"] + master_df["5 bedrooms"] + master_df["6 or more bedrooms"]\n# Column summing house composition\nmaster_df["Sum of House Composition"] = master_df["Couple with no children"] + master_df["Couple with children"] + master_df["One parent with children"] + master_df["Sole person household"] + master_df["Group household"]\nmaster_df\n'

# Output

In [11]:
# Write master dataframe to CSV
master_df.to_csv("Output/master_data.csv")


In [12]:
# Write postcode / state dataframe to CSV
master_df["State"].to_csv("Output/postcodes.csv")